In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from keras.layers import Dense, InputLayer
import tensorflow_model_optimization as tfmot

In [2]:
# Modified this dataset:
# https://github.com/AjinkyaChavan9/RGB-Color-Classifier-with-Deep-Learning-using-Keras-and-Tensorflow/blob/master/Dataset/final_data.csv

df = pd.read_csv("cleaned_data.csv")

color_codes = {
    "White": 0,
    "Red": 1,
    "Yellow": 2,
    "Green": 3,
    "Brown": 4,
    "Blue": 5,
    "Pink": 6,
    "Black": 7,
}
df["label"] = df["label"].map(lambda x: color_codes.get(x))

df.groupby(["label"]).count()

,red,green,blue
label,,,
0,151,151,151
1,500,500,500
2,322,322,322
3,1503,1503,1503
4,376,376,376
5,1164,1164,1164
6,578,578,578
7,113,113,113


In [3]:
train = df.sample(frac=0.8, random_state=8)
test = df.drop(train.index)

x_train = train[["red", "green", "blue"]]
y_train = train["label"]

x_test = test[["red", "green", "blue"]]
y_test = test["label"]

In [4]:
model = Sequential()

model.add(InputLayer(input_shape=(3,)))

model.add(Dense(7, activation='relu'))

model.add(Dense(8, activation='softmax'))

model.summary()

2021-08-20 22:38:17.511644: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 7)                 28        
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 64        
Total params: 92
Trainable params: 92
Non-trainable params: 0
_________________________________________________________________


2021-08-20 22:38:17.521190: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-20 22:38:17.521733: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-20 22:38:17.523319: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-08-20 22:38:17.524098: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [5]:
model.compile(optimizer="adam",
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=500)

2021-08-20 22:38:22.618905: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/500
118/118 [==============================] - 1s 2ms/step - loss: 99.0738 - accuracy: 0.1338
Epoch 2/500
118/118 [==============================] - 0s 2ms/step - loss: 45.3625 - accuracy: 0.3502
Epoch 3/500
118/118 [==============================] - 0s 2ms/step - loss: 20.6770 - accuracy: 0.3550
Epoch 4/500
118/118 [==============================] - 0s 2ms/step - loss: 7.0757 - accuracy: 0.4488
Epoch 5/500
118/118 [==============================] - 0s 2ms/step - loss: 3.8064 - accuracy: 0.6118
Epoch 6/500
118/118 [==============================] - 0s 2ms/step - loss: 2.9219 - accuracy: 0.7111
Epoch 7/500
118/118 [==============================] - 0s 1ms/step - loss: 2.4272 - accuracy: 0.7387
Epoch 8/500
118/118 [==============================] - 0s 1ms/step - loss: 2.0056 - accuracy: 0.7480
Epoch 9/500
118/118 [==============================] - 0s 1ms/step - loss: 1.7074 - accuracy: 0.7477
Epoch 10/500
118/118 [==============================] - 0s 2ms/step - loss: 1.4281 - acc

In [6]:
_, baseline_model_accuracy = model.evaluate(
    x_test, y_test, verbose=0)
print('Baseline test accuracy:', baseline_model_accuracy)


Baseline test accuracy: 0.9032943844795227


In [7]:
import numpy as np
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set. 

num_images = x_train.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_for_pruning.summary()
logdir = "."

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.fit(x_train, y_train,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)


/home/shi/.virtualenvs/tensorflow/lib/python3.9/site-packages/keras/engine/base_layer.py:2223: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
prune_low_magnitude_dense (P (None, 7)                 51        
_________________________________________________________________
prune_low_magnitude_dense_1  (None, 8)                 122       
Total params: 173
Trainable params: 92
Non-trainable params: 81
_________________________________________________________________


2021-08-20 22:40:14.860010: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-08-20 22:40:14.860038: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-08-20 22:40:14.860629: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1614] Profiler found 1 GPUs
2021-08-20 22:40:14.860902: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so.11.2'; dlerror: libcupti.so.11.2: cannot open shared object file: No such file or directory
2021-08-20 22:40:15.200630: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-08-20 22:40:15.200793: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1748] CUPTI activity buffer flushed
2021-08-20 22:40:15.276072: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Epoch 1/2


/home/shi/.virtualenvs/tensorflow/lib/python3.9/site-packages/keras/backend.py:4906: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(


 2/27 [=>............................] - ETA: 5s - loss: 0.2459 - accuracy: 0.9102 

2021-08-20 22:40:16.815183: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-08-20 22:40:16.815221: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-08-20 22:40:17.001919: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-08-20 22:40:17.002247: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1748] CUPTI activity buffer flushed


21/27 [======================>.......] - ETA: 0s - loss: 0.2409 - accuracy: 0.9129

2021-08-20 22:40:17.063235: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:673]  GpuTracer has collected 91 callback api events and 88 activity events. 
2021-08-20 22:40:17.066833: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-08-20 22:40:17.073077: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: ./train/plugins/profile/2021_08_20_22_40_17

2021-08-20 22:40:17.076619: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to ./train/plugins/profile/2021_08_20_22_40_17/daniel-ms7c37.trace.json.gz
2021-08-20 22:40:17.086357: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: ./train/plugins/profile/2021_08_20_22_40_17

2021-08-20 22:40:17.087504: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for memory_profile.json.gz to ./train/plugins/profile/2021_08_20_22_40_17/daniel-ms7c37.memory_profile

27/27 [==============================] - 2s 21ms/step - loss: 0.2375 - accuracy: 0.9144 - val_loss: 0.2988 - val_accuracy: 0.9178
Epoch 2/2
27/27 [==============================] - 0s 4ms/step - loss: 0.2368 - accuracy: 0.9162 - val_loss: 0.2899 - val_accuracy: 0.9204


In [10]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
pruned_tflite_model = converter.convert()
pruned_tflite_file = "color_classifier.tflite"

with open(pruned_tflite_file, 'wb') as f:
  f.write(pruned_tflite_model)


INFO:tensorflow:Assets written to: /tmp/tmp9tjyepsi/assets


INFO:tensorflow:Assets written to: /tmp/tmp9tjyepsi/assets
2021-08-20 22:44:46.478672: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-20 22:44:46.479052: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2021-08-20 22:44:46.479176: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2021-08-20 22:44:46.479462: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-20 22:44:46.479818: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-20 22:44:46.480142: I t

In [8]:
model.save("color_classifier.h5")

array([2])